In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt # general plotting
import cartopy.crs as ccrs # plot on maps, better than the Basemap module
import re
from scipy.interpolate import interp1d

In [2]:
ds = xr.open_dataset(".././initial_GEOSChem_rst.4x5_benchmark.nc")
ds # same as print(ds) in IPython/Jupyter environment

<xarray.Dataset>
Dimensions:       (lat: 46, lev: 72, lon: 72, time: 1)
Coordinates:
  * lon           (lon) float64 -180.0 -175.0 -170.0 -165.0 -160.0 -155.0 ...
  * lat           (lat) float64 -89.0 -86.0 -82.0 -78.0 -74.0 -70.0 -66.0 ...
  * lev           (lev) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 ...
  * time          (time) datetime64[ns] 2013-07-01
Data variables:
    AREA          (lat, lon) float64 ...
    SPC_RCOOH     (time, lev, lat, lon) float32 ...
    SPC_O2        (time, lev, lat, lon) float32 ...
    SPC_N2        (time, lev, lat, lon) float32 ...
    SPC_MOH       (time, lev, lat, lon) float32 ...
    SPC_H2        (time, lev, lat, lon) float32 ...
    SPC_O         (time, lev, lat, lon) float32 ...
    SPC_HO2       (time, lev, lat, lon) float32 ...
    SPC_O1D       (time, lev, lat, lon) float32 ...
    SPC_OH        (time, lev, lat, lon) float32 ...
    SPC_MO2       (time, lev, lat, lon) float32 ...
    SPC_MCO3      (time, lev, lat, lon) float32 ...
  

In [14]:
lev = np.linspace(1,132,num =132,dtype = 'float64')
molec = list()
i = 0
pattern = "(SPC_\w+)"
for i, line in enumerate(open('../molecular_species.txt')):
    for match in re.finditer(pattern,line):
        #molec.append(match)
        molec.append(match.group())


In [15]:
dataset = xr.Dataset()
dataset.coords['time'] = ds.coords['time']
dataset.coords['lev'] = (('lev'),lev)
dataset.coords['lat'] = ds.coords['lat'] 
dataset.coords['lon'] = ds.coords['lon']
dataset.attrs['title'] = 'GEOS-5 132 lvl restart'
dataset.attrs['history'] = 'created by Ada Shaw with initial_GEOSChem_rst.4x5_benchmark.nc'
dataset.attrs['format'] = "NetCDF-4"
dataset.attrs['conventions'] = 'COARDS'

In [19]:
for tick_molec in range(len(molec)):
    dr = ds[molec[tick_molec]].values
    #dr.shape # time:1, lev:72, lat:46, lon:72
    #initialize x_interp array to be 1x132x46x72
    type(dr)
    x_interp = np.ndarray(shape=(1,132,46,72), dtype=float, order='F')
    for tick_lat in range(46):
        for tick_lon in range(72):
            xo = dr[0,0,tick_lat,tick_lon]
            xf = dr[0,71,tick_lat,tick_lon]
            x_interp[0,:,tick_lat,tick_lon]=np.linspace(xo,xf,num=132,dtype = float)
    dataset[molec[tick_molec]] = (('time','lev','lat','lon'),x_interp)

In [21]:
dataset

<xarray.Dataset>
Dimensions:       (lat: 46, lev: 132, lon: 72, time: 1)
Coordinates:
  * time          (time) datetime64[ns] 2013-07-01
  * lev           (lev) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 ...
  * lat           (lat) float64 -89.0 -86.0 -82.0 -78.0 -74.0 -70.0 -66.0 ...
  * lon           (lon) float64 -180.0 -175.0 -170.0 -165.0 -160.0 -155.0 ...
Data variables:
    SPC_RCOOH     (time, lev, lat, lon) float64 1e-20 1e-20 1e-20 1e-20 ...
    SPC_O2        (time, lev, lat, lon) float64 0.2095 0.2095 0.2095 0.2095 ...
    SPC_N2        (time, lev, lat, lon) float64 0.7808 0.7808 0.7808 0.7808 ...
    SPC_MOH       (time, lev, lat, lon) float64 2e-09 2e-09 2e-09 2e-09 ...
    SPC_H2        (time, lev, lat, lon) float64 5e-07 5e-07 5e-07 5e-07 ...
    SPC_O         (time, lev, lat, lon) float64 1e-30 1e-30 1e-30 1e-30 ...
    SPC_HO2       (time, lev, lat, lon) float64 1.418e-16 1.418e-16 ...
    SPC_O1D       (time, lev, lat, lon) float64 1e-30 1e-30 1e-30 1e-30 ...
  

In [24]:
dataset.to_netcdf('132_lvl_rst.nc','w')